# Import Libraries

In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
import pandas as pd
import numpy as np
import networkx as nx

# Using the Disparity Filter

In [3]:
#import the disparity filter module
from Backbones import disparity_filter as disf

#read edge list from csv file
edges_list = pd.read_csv('../Datasets/lesmis.csv')

#create graph from the edge list
G = nx.from_pandas_edgelist(edges_list, edge_attr='weight', create_using=nx.Graph())

#apply the disparity filter algorithm
backbone = disf.disparity_filter(G)

#create an edge list from the result graph with the scores
df_backbone = nx.to_pandas_edgelist(backbone)

#save the edge list to visualize later
df_backbone.to_csv('../Results/lesmis_df_backbone.csv', index=False)

# Using the High Salience Skeleton

In [4]:
#import the high salience skeleton module
from Backbones.high_salience_skeleton import read, high_salience_skeleton as hss

#read edge list from csv file using the high salience skeleton module
table, nnodes, nnedges = read("../Datasets/lesmis.csv", "weight", sep=',', consider_self_loops=False, triangular_input = True, undirected=True) 

#apply the high salience skeleton algorithm
hss_backbone = hss(table, return_self_loops=False, undirected=True)

#save the edge list to visualize later
hss_backbone.to_csv('../Results/lesmis_hss_backbone.csv', index=False)

# Using the Distance Closure

In [5]:
#import the distance closure modules
from distanceclosure.utils import prox2dist, dist2prox
from distanceclosure.distance import pairwise_proximity
import distanceclosure as dc

#read edge list from csv file
edges_list = pd.read_csv('../Datasets/lesmis.csv')

#create graph from the edge list
labeled_G = nx.from_pandas_edgelist(edges_list, edge_attr='weight', create_using=nx.Graph())

#convert node labels to integers and store the labels as attributes and get the label used for mapping later
G = nx.convert_node_labels_to_integers(G, label_attribute='name')
mapping_lables = nx.get_node_attributes(G, name='name')

#create the adjacency matrix of the graph
W = nx.adjacency_matrix(G).todense()

#calculate the proximity matrix using the weighted jaccard algorithm
P = pairwise_proximity(W, metric='jaccard_weighted')

#convert the proximity matrix to a distance matrix
D = np.vectorize(prox2dist)(P)

#create a distance graph from the distance matrix containing only the edges observed in the original network
DG = nx.from_numpy_matrix(D)
for u,v in DG.edges():
    edge = (u,v)
    if edge not in G.edges():
        DG.remove_edge(u, v)

#apply the distance closure algorithm to obtain the metric and ultrametric backbones
m_backbone = dc.distance_closure(DG, kind='metric', weight='weight', only_backbone=True)
um_backbone = dc.distance_closure(DG, kind='ultrametric', weight='weight', only_backbone=True)

#relabel the graphs with the original labels
m_backbone = nx.relabel_nodes(m_backbone, mapping_lables)
um_backbone = nx.relabel_nodes(um_backbone, mapping_lables)

#create an edge list from the result graph with a boolean value that shows if an edge is metric or not
metric_backbone = nx.to_pandas_edgelist(m_backbone)
ultrametric_backbone = nx.to_pandas_edgelist(um_backbone)

#save the edge list to visualize later
metric_backbone.to_csv('../Results/lesmis_metric_backbone.csv', index=False)
ultrametric_backbone.to_csv('../Results/lesmis_ultrametric_backbone.csv', index=False)